In [5]:
import os
import numpy as np
import subprocess
from train.global_p import match_label_video
from os import path

In [2]:
rnk_path1 = '../gordo_retrieval/triplet_best'
rnk_path2 = '../gordo_retrieval/gordo_multires'
eval_file = '../gordo_retrieval/datasets/evaluation/compute_ap'
lab_root = '../gordo_retrieval/datasets/fourviere_video/lab'
match = match_label_video

In [7]:
def f_2_name(fname):
    return path.splitext(path.basename(fname))[0]
def f_2_lab(fname):
    return match(fname)

In [8]:
clicide_qnames = [f_2_name(fname)[:-len('_query')] for fname in os.listdir(lab_root) if fname.endswith('_query.txt')]

In [9]:
def ap_rnk(rnk_file):
    cmd = "{0} {1} {2}".format(eval_file, os.path.join(lab_root, f_2_name(rnk_file)), rnk_file)
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    map_ = float(p.stdout.readlines()[0])
    p.wait()
    return map_

def prec1_rnk(rnk_file):
    with open(rnk_file) as f:
        res = int(f.readline().split('-')[0] == f_2_lab(rnk_file))
    return res

In [10]:
results = {}
ap_threshold = 0.2
for rnk_path in (rnk_path1, rnk_path2):
    results[rnk_path] = {}
    for rnk_file in os.listdir(rnk_path):
        name = f_2_name(rnk_file)
        if name not in clicide_qnames:
            continue

        rnk_file = os.path.join(rnk_path, rnk_file)
        ap = ap_rnk(rnk_file)
        prec1 = prec1_rnk(rnk_file)
        results[rnk_path][name] = (ap, prec1, ap < ap_threshold)
    print('Analysis of: {0}'.format(rnk_path))
    print('mean AP: {0}'.format(np.mean([results[rnk_path][name][0] for name in results[rnk_path]])))
    print('mean Prec1: {0}'.format(np.mean([results[rnk_path][name][1] for name in results[rnk_path]])))
    print('#files with AP<{0}: {1}'.format(ap_threshold, sum(results[rnk_path][name][2] for name in results[rnk_path])))
    print('mean AP of these files: {0}'.format(np.mean([results[rnk_path][name][0] for name in results[rnk_path] if results[rnk_path][name][2]])))

Analysis of: ../gordo_retrieval/triplet_best
mean AP: 0.427403468782
mean Prec1: 0.440705128205
#files with AP<0.2: 294
mean AP of these files: 0.0413138146939
Analysis of: ../gordo_retrieval/gordo_multires
mean AP: 0.602620064989
mean Prec1: 0.732371794872
#files with AP<0.2: 149
mean AP of these files: 0.055746111094


In [11]:
print('Difficult images for both')
for name in results[rnk_path1]:
    if results[rnk_path1][name][2] and results[rnk_path2][name][2]:
        print(name, results[rnk_path1][name], results[rnk_path2][name])
print('Difficult images for us, not them')
for name in results[rnk_path1]:
    if results[rnk_path1][name][2] and not results[rnk_path2][name][2]:
        print(name, results[rnk_path1][name], results[rnk_path2][name])

Difficult images for both
('A_007-u3_A-119.36', (0.0172414, 0, True), (0.00100604, 0, True))
('B_059-u3_B-922.68', (0.0208333, 0, True), (0.0416667, 0, True))
('B_021-u3_B-426.92', (0.00505051, 0, True), (0.125, 0, True))
('C_006-u3_C-42.9', (0.0511882, 0, True), (0.00646858, 0, True))
('A_026-u3_A-436.48', (0.0521394, 0, True), (0.00737842, 0, True))
('B_054-u3_B-738.08', (0.0566445, 0, True), (0.17994, 0, True))
('A_015-u3_A-282.84', (0.00104822, 0, True), (0.166667, 0, True))
('B_073-u3_B-967.32', (0.0635384, 0, True), (0.0557009, 0, True))
('A_018-u3_A-403.8', (0.00279233, 0, True), (0.00219846, 0, True))
('A_025-u3_A-458.72', (0.00874603, 0, True), (0.00331445, 0, True))
('B_055-u3_B-753.52', (0.0774524, 0, True), (0.0785323, 0, True))
('C_004-u3_C-5.19', (0.0243687, 0, True), (0.0243445, 0, True))
('C_005-u3_C-33.44', (0.00823116, 0, True), (0.0845655, 0, True))
('D_006-u3_D-42.23', (0.00330338, 0, True), (0.196429, 0, True))
('B_060-u3_B-936.36', (0.0131579, 0, True), (0.166667,

In [12]:
print('Easy images for both')
for name in results[rnk_path1]:
    if results[rnk_path1][name][0] >= 1. and results[rnk_path2][name][0] >= 1.:
        print(name, results[rnk_path1][name], results[rnk_path2][name])

Easy images for both
('B_064-u3_B-1073.88', (1.0, 1, False), (1.0, 1, False))
('B_064-u3_B-1073.36', (1.0, 1, False), (1.0, 1, False))
('D_036-u3_D-586.89', (1.0, 1, False), (1.0, 1, False))
('A_021-u3_A-400.08', (1.0, 1, False), (1.0, 1, False))
('B_060-u3_B-932.76', (1.0, 1, False), (1.0, 1, False))
('D_083-u3_D-248.6', (1.0, 1, False), (1.0, 1, False))
('A_011-u3_A-177.88', (1.0, 1, False), (1.0, 1, False))
('D_083-u3_D-246.28', (1.0, 1, False), (1.0, 1, False))
('D_083-u3_D-244.4', (1.0, 1, False), (1.0, 1, False))
('A_011-u3_A-199.88', (1.0, 1, False), (1.0, 1, False))
('D_056-u3_D-825.3', (1.0, 1, False), (1.0, 1, False))
('D_036-u3_D-606.55', (1.0, 1, False), (1.0, 1, False))
('A_028-u3_A-426.88', (1.0, 1, False), (1.0, 1, False))
('B_038-u3_B-708.04', (1.0, 1, False), (1.0, 1, False))
('A_006-u3_A-80.0', (1.0, 1, False), (1.0, 1, False))
('B_064-u3_B-1077.24', (1.0, 1, False), (1.0, 1, False))
('B_038-u3_B-707.12', (1.0, 1, False), (1.0, 1, False))
('B_064-u3_B-1072.52', (1.0, 